In [76]:
%matplotlib inline
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestClassifier
import  sklearn.linear_model as linearModel
from sklearn.decomposition import PCA
from sklearn import preprocessing
import numpy as np
import seaborn 
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings('ignore')
import timeit
import pymc3 as pm
import matplotlib.lines as mlines
from sklearn import svm
np.random.seed(1)
from itertools import product, combinations

In [ ]:
import bridgeSampling as bs  # it contains a method to estimate the marginal likelihood according to the Bridge Sampling approach
import utilFunctions as ut   # it has different methods to handle and plot data 
import BayesianModels as bm  # it has methods to build and train bayesian model (Logistic Regression and Neural Nets) 
import experiments as exp 

In [ ]:
Data, ID, Class_col, Classes = ut.initialize_data(survey='GAIA', sepColumns_=',', sepHeader_=',')


Running GAIA


In [ ]:
Data[Class_col].value_counts()

In [ ]:
Data.columns

In [ ]:
def sampleBiasSelection(Data, **kwargs):
      
    Data = Data[(Data[kwargs['name_class_col']] == kwargs['class_1']) | 
          (Data[kwargs['name_class_col']] == kwargs['class_2'])]  
    
    label = 1*(Data[kwargs['name_class_col']] == kwargs['class_1'])
    DataBiased = Data.copy()
    
    del DataBiased[kwargs['name_class_col']]
    del DataBiased[kwargs['id_col']]

    clf = RandomForestClassifier(max_depth=kwargs['deep_Max'], random_state=0)

    try:
        DataBiased = DataBiased.replace('\n', '', regex = True).replace('null', '0.0', regex = True).apply(pd.to_numeric, errors ='ignore')
    except:
        print('Error in replace')

    clf.fit(DataBiased, label)

    Pred2 =  clf.predict_proba(DataBiased)
    Pred =  clf.predict(DataBiased)
    print('Acc:', accuracy_score(Pred, label))

    Data['Pred'] = Pred2[:,0].tolist()
    Data['Pred2'] = Pred2[:,1].tolist()
    
    Data['h'] = 1 -  Data['Pred']*Data['Pred'] - Data['Pred2']*Data['Pred2']
    factor = kwargs['T']*Data['h']
    Data['e'] = np.exp(-factor)
    Data['u'] = np.random.uniform(0, 1, Data.shape[0])
    
    #print(Data['e'])
    #print(factor)
    #print(Data['u'])
  
    Data_test  = Data[(Data['e']<= Data['u'])]
    Data_train = Data[(Data['e']> Data['u'])]

    
    label_train = Data_train[kwargs['name_class_col']]
    label_test = Data_test[kwargs['name_class_col']]
    del Data_train[kwargs['name_class_col']]
    del Data_test[kwargs['name_class_col']] 

    print('Shape training: ', Data_train.shape)
    print('Shape testing: ', Data_test.shape)
    return Data_train, Data_test, label_train, label_test

In [ ]:
kwargs = {
          "name_class_col": 'Class', 
          "class_1" : 'RRAB', 
          "class_2": 'RRC',
           "id_col" : 'ID',
           "deep_Max" : 5, 
           "T": 2
        }

In [ ]:
Data_train, Data_test, label_train, label_test = sampleBiasSelection(Data, **kwargs)

In [ ]:
Data_test.Pred.hist()

In [ ]:
Data_train.Pred.hist()

In [ ]:
#sets up the axis and gets histogram data
x = Data_train.Pred
y = Data_test.Pred
x_w = np.empty(x.shape)
x_w.fill(1/x.shape[0])
y_w = np.empty(y.shape)
y_w.fill(1/y.shape[0])
bins = np.linspace(0, 1, 10)
plt.hist([x, y], bins, weights=[x_w, y_w], label=['training set', 'testing set'])
plt.legend(loc='upper right')
plt.xlabel('soft predict')
plt.ylabel('normalized frequency')
plt.show()

In [ ]:
Data_train.to_csv('Train_GAIA_RR.csv')

In [ ]:
Data_test.to_csv('Test_GAIA_RR.csv')